In [87]:
from nba_api.stats.endpoints import leaguegamefinder
import pandas as pd

mavs_id = '1610612742'
lakers_id = '1610612747'

gamefinder = leaguegamefinder.LeagueGameFinder(team_id_nullable= mavs_id)
games = gamefinder.get_data_frames()[0]

games_vs_lakers = games[games['MATCHUP'].str.contains("LAL")]

games_vs_lakers = games_vs_lakers[games_vs_lakers['SEASON_ID'].str.startswith('2')]


print(games_vs_lakers.head())

    SEASON_ID     TEAM_ID TEAM_ABBREVIATION         TEAM_NAME     GAME_ID  \
21      22024  1610612742               DAL  Dallas Mavericks  0022400504   
131     22023  1610612742               DAL  Dallas Mavericks  0022300577   
150     22023  1610612742               DAL  Dallas Mavericks  0022300306   
158     22023  1610612742               DAL  Dallas Mavericks  0022300238   
193     22022  1610612742               DAL  Dallas Mavericks  0022201054   

      GAME_DATE      MATCHUP WL  MIN  PTS  ...  FT_PCT  OREB  DREB  REB  AST  \
21   2025-01-07  DAL vs. LAL  W  238  118  ...   0.714    11    33   44   29   
131  2024-01-17    DAL @ LAL  L  240  110  ...   0.833    11    32   43   26   
150  2023-12-12  DAL vs. LAL  W  239  127  ...   0.857    12    24   36   28   
158  2023-11-22    DAL @ LAL  W  240  104  ...   0.750    11    37   48   21   
193  2023-03-17    DAL @ LAL  W  240  111  ...   0.789     7    28   35   27   

     STL  BLK  TOV  PF  PLUS_MINUS  
21     7    0   10 

In [89]:
from nba_api.stats.endpoints import playergamelog
import pandas as pd

# Luka Doncic Player ID
luka_id = '1629029'

# Fetch Luka's game logs
luka_stats = playergamelog.PlayerGameLog(player_id=luka_id, season="2023-24")
luka_df = luka_stats.get_data_frames()[0]

# Display his recent games
print(luka_df[['GAME_DATE', 'MATCHUP', 'PTS', 'REB', 'AST', 'PLUS_MINUS']].head())


      GAME_DATE      MATCHUP  PTS  REB  AST  PLUS_MINUS
0  APR 10, 2024    DAL @ MIA   29    9    9           7
1  APR 09, 2024    DAL @ CHA   39   12   10          22
2  APR 07, 2024  DAL vs. HOU   37    9   12          20
3  APR 04, 2024  DAL vs. ATL   25   12    8          21
4  APR 02, 2024    DAL @ GSW   30   12   11           9


In [91]:
# Adjust Mavericks' stats (remove Luka's contributions)
games_vs_lakers.loc[:, 'PTS'] -= luka_df['PTS'].mean()
games_vs_lakers.loc[:, 'AST'] -= luka_df['AST'].mean()

# Adjust Lakers' stats (add Luka's contributions)
games_vs_lakers.loc[:, 'LAL_PTS'] += luka_df['PTS'].mean()
games_vs_lakers.loc[:, 'LAL_AST'] += luka_df['AST'].mean()


C:\Users\ambyb\AppData\Local\Temp\ipykernel_22740\832086727.py:2: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[ 84.14285714  76.14285714  93.14285714  70.14285714  77.14285714
  74.14285714  85.14285714  90.14285714  50.14285714  94.14285714
  75.14285714  70.14285714  74.14285714  81.14285714  81.14285714
  80.14285714  61.14285714  80.14285714  76.14285714  63.14285714
  69.14285714  79.14285714  59.14285714  68.14285714  96.14285714
  67.14285714  64.14285714  88.14285714  88.14285714  67.14285714
  75.14285714  58.14285714  56.14285714  69.14285714  54.14285714
  86.14285714  66.14285714  68.14285714 106.14285714  73.14285714
  76.14285714  89.14285714  47.14285714  65.14285714  55.14285714
  65.14285714  74.14285714  59.14285714  57.14285714  36.14285714
  48.14285714  57.14285714  75.14285714  67.14285714  61.14285714
  62.14285714  60.14285714  66.14285714  73.14285714  65.14285714
  74.14285714  66.142857

KeyError: 'LAL_PTS'

In [93]:
print(games_vs_lakers.columns)
print(games_vs_lakers.head())


Index(['SEASON_ID', 'TEAM_ID', 'TEAM_ABBREVIATION', 'TEAM_NAME', 'GAME_ID',
       'GAME_DATE', 'MATCHUP', 'WL', 'MIN', 'PTS', 'FGM', 'FGA', 'FG_PCT',
       'FG3M', 'FG3A', 'FG3_PCT', 'FTM', 'FTA', 'FT_PCT', 'OREB', 'DREB',
       'REB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PLUS_MINUS'],
      dtype='object')
    SEASON_ID     TEAM_ID TEAM_ABBREVIATION         TEAM_NAME     GAME_ID  \
21      22024  1610612742               DAL  Dallas Mavericks  0022400504   
131     22023  1610612742               DAL  Dallas Mavericks  0022300577   
150     22023  1610612742               DAL  Dallas Mavericks  0022300306   
158     22023  1610612742               DAL  Dallas Mavericks  0022300238   
193     22022  1610612742               DAL  Dallas Mavericks  0022201054   

      GAME_DATE      MATCHUP WL  MIN        PTS  ...  FT_PCT  OREB  DREB  REB  \
21   2025-01-07  DAL vs. LAL  W  238  84.142857  ...   0.714    11    33   44   
131  2024-01-17    DAL @ LAL  L  240  76.142857  ...   0.833    1

In [95]:
# Create two dataframes: one for Mavericks stats, one for Lakers stats
mavs_games = games_vs_lakers[games_vs_lakers['TEAM_ID'] == 1610612742]  # Mavericks team ID
lakers_games = games_vs_lakers[games_vs_lakers['TEAM_ID'] == 1610612747]  # Lakers team ID

# Rename columns to indicate team-specific stats
mavs_games = mavs_games.rename(columns={'PTS': 'MAVS_PTS', 'REB': 'MAVS_REB', 'AST': 'MAVS_AST'})
lakers_games = lakers_games.rename(columns={'PTS': 'LAKERS_PTS', 'REB': 'LAKERS_REB', 'AST': 'LAKERS_AST'})

# Merge Mavericks and Lakers data on game date to align the stats
games_vs_lakers = mavs_games.merge(lakers_games, on='GAME_ID', suffixes=('_MAVS', '_LAKERS'))


In [137]:
from nba_api.stats.endpoints import leaguegamefinder
import pandas as pd

# Fetch Mavericks games
gamefinder = leaguegamefinder.LeagueGameFinder(team_id_nullable=1610612742)
mavs_data = gamefinder.get_data_frames()[0]

print(f"Mavericks games retrieved: {len(mavs_data)}")
print(mavs_data.head())


Mavericks games retrieved: 3830
  SEASON_ID     TEAM_ID TEAM_ABBREVIATION         TEAM_NAME     GAME_ID  \
0     22024  1610612742               DAL  Dallas Mavericks  0022400812   
1     22024  1610612742               DAL  Dallas Mavericks  0022400803   
2     22024  1610612742               DAL  Dallas Mavericks  0022400786   
3     22024  1610612742               DAL  Dallas Mavericks  0022400782   
4     22024  1610612742               DAL  Dallas Mavericks  0022400762   

    GAME_DATE      MATCHUP WL  MIN  PTS  ...  FT_PCT  OREB  DREB  REB  AST  \
0  2025-02-23    DAL @ GSW  L  240  102  ...   0.684     7    34   41   24   
1  2025-02-21  DAL vs. NOP  W  240  111  ...   0.778     8    39   47   28   
2  2025-02-13  DAL vs. MIA  W  242  118  ...   0.813    11    29   40   28   
3  2025-02-12  DAL vs. GSW  W  240  111  ...   0.800     9    30   39   20   
4  2025-02-10  DAL vs. SAC  L  265  128  ...   0.625     5    28   33   31   

   STL  BLK  TOV  PF  PLUS_MINUS  
0    7    4  

In [145]:
print(games_vs_lakers.columns.tolist())  # Available columns
print(games_vs_lakers.head())  # First few rows
print(f"Total games: {len(games_vs_lakers)}")  # Ensure data exists



['SEASON_ID_MAVS', 'TEAM_ID_MAVS', 'TEAM_ABBREVIATION_MAVS', 'TEAM_NAME_MAVS', 'GAME_ID', 'GAME_DATE_MAVS', 'MATCHUP_MAVS', 'WL_MAVS', 'MIN_MAVS', 'MAVS_PTS', 'FGM_MAVS', 'FGA_MAVS', 'FG_PCT_MAVS', 'FG3M_MAVS', 'FG3A_MAVS', 'FG3_PCT_MAVS', 'FTM_MAVS', 'FTA_MAVS', 'FT_PCT_MAVS', 'OREB_MAVS', 'DREB_MAVS', 'MAVS_REB', 'MAVS_AST', 'STL_MAVS', 'BLK_MAVS', 'TOV_MAVS', 'PF_MAVS', 'PLUS_MINUS_MAVS', 'SEASON_ID_LAKERS', 'TEAM_ID_LAKERS', 'TEAM_ABBREVIATION_LAKERS', 'TEAM_NAME_LAKERS', 'GAME_DATE_LAKERS', 'MATCHUP_LAKERS', 'WL_LAKERS', 'MIN_LAKERS', 'LAKERS_PTS', 'FGM_LAKERS', 'FGA_LAKERS', 'FG_PCT_LAKERS', 'FG3M_LAKERS', 'FG3A_LAKERS', 'FG3_PCT_LAKERS', 'FTM_LAKERS', 'FTA_LAKERS', 'FT_PCT_LAKERS', 'OREB_LAKERS', 'DREB_LAKERS', 'LAKERS_REB', 'LAKERS_AST', 'STL_LAKERS', 'BLK_LAKERS', 'TOV_LAKERS', 'PF_LAKERS', 'PLUS_MINUS_LAKERS']
Empty DataFrame
Columns: [SEASON_ID_MAVS, TEAM_ID_MAVS, TEAM_ABBREVIATION_MAVS, TEAM_NAME_MAVS, GAME_ID, GAME_DATE_MAVS, MATCHUP_MAVS, WL_MAVS, MIN_MAVS, MAVS_PTS, FGM_

In [141]:
mavs_games = games_vs_lakers[['GAME_ID', 'TEAM_NAME_MAVS', 'MAVS_PTS']] if 'TEAM_NAME_MAVS' in games_vs_lakers.columns else pd.DataFrame()
lakers_games = games_vs_lakers[['GAME_ID', 'TEAM_NAME_LAKERS', 'LAKERS_PTS']] if 'TEAM_NAME_LAKERS' in games_vs_lakers.columns else pd.DataFrame()


In [ ]:
print(f"Mavericks games count: {len(mavs_games)}")
print(mavs_games.head())

print(f"Lakers games count: {len(lakers_games)}")
print(lakers_games.head())

Mavericks games count: 0
Empty DataFrame
Columns: [SEASON_ID_MAVS, TEAM_ID_MAVS, TEAM_ABBREVIATION_MAVS, TEAM_NAME_MAVS, GAME_ID, GAME_DATE_MAVS, MATCHUP_MAVS, WL_MAVS, MIN_MAVS, MAVS_PTS, FGM_MAVS, FGA_MAVS, FG_PCT_MAVS, FG3M_MAVS, FG3A_MAVS, FG3_PCT_MAVS, FTM_MAVS, FTA_MAVS, FT_PCT_MAVS, OREB_MAVS, DREB_MAVS, MAVS_REB, MAVS_AST, STL_MAVS, BLK_MAVS, TOV_MAVS, PF_MAVS, PLUS_MINUS_MAVS, SEASON_ID_LAKERS, TEAM_ID_LAKERS, TEAM_ABBREVIATION_LAKERS, TEAM_NAME_LAKERS, GAME_DATE_LAKERS, MATCHUP_LAKERS, WL_LAKERS, MIN_LAKERS, LAKERS_PTS, FGM_LAKERS, FGA_LAKERS, FG_PCT_LAKERS, FG3M_LAKERS, FG3A_LAKERS, FG3_PCT_LAKERS, FTM_LAKERS, FTA_LAKERS, FT_PCT_LAKERS, OREB_LAKERS, DREB_LAKERS, LAKERS_REB, LAKERS_AST, STL_LAKERS, BLK_LAKERS, TOV_LAKERS, PF_LAKERS, PLUS_MINUS_LAKERS]
Index: []

[0 rows x 55 columns]
Lakers games count: 0
Empty DataFrame
Columns: [SEASON_ID_MAVS, TEAM_ID_MAVS, TEAM_ABBREVIATION_MAVS, TEAM_NAME_MAVS, GAME_ID, GAME_DATE_MAVS, MATCHUP_MAVS, WL_MAVS, MIN_MAVS, MAVS_PTS, FGM_MAVS,

In [ ]:
#print("Unique matchups in games_vs_lakers:")
#print(games_vs_lakers['MATCHUP'].unique())


print(games_vs_lakers.columns.tolist())

['SEASON_ID_MAVS', 'TEAM_ID_MAVS', 'TEAM_ABBREVIATION_MAVS', 'TEAM_NAME_MAVS', 'GAME_ID', 'GAME_DATE_MAVS', 'MATCHUP_MAVS', 'WL_MAVS', 'MIN_MAVS', 'MAVS_PTS', 'FGM_MAVS', 'FGA_MAVS', 'FG_PCT_MAVS', 'FG3M_MAVS', 'FG3A_MAVS', 'FG3_PCT_MAVS', 'FTM_MAVS', 'FTA_MAVS', 'FT_PCT_MAVS', 'OREB_MAVS', 'DREB_MAVS', 'MAVS_REB', 'MAVS_AST', 'STL_MAVS', 'BLK_MAVS', 'TOV_MAVS', 'PF_MAVS', 'PLUS_MINUS_MAVS', 'SEASON_ID_LAKERS', 'TEAM_ID_LAKERS', 'TEAM_ABBREVIATION_LAKERS', 'TEAM_NAME_LAKERS', 'GAME_DATE_LAKERS', 'MATCHUP_LAKERS', 'WL_LAKERS', 'MIN_LAKERS', 'LAKERS_PTS', 'FGM_LAKERS', 'FGA_LAKERS', 'FG_PCT_LAKERS', 'FG3M_LAKERS', 'FG3A_LAKERS', 'FG3_PCT_LAKERS', 'FTM_LAKERS', 'FTA_LAKERS', 'FT_PCT_LAKERS', 'OREB_LAKERS', 'DREB_LAKERS', 'LAKERS_REB', 'LAKERS_AST', 'STL_LAKERS', 'BLK_LAKERS', 'TOV_LAKERS', 'PF_LAKERS', 'PLUS_MINUS_LAKERS']


In [67]:
print("Available columns in games_vs_lakers:")
print(games_vs_lakers.columns.tolist())  # List all column names


Available columns in games_vs_lakers:
['SEASON_ID', 'TEAM_ID', 'TEAM_ABBREVIATION', 'TEAM_NAME', 'GAME_ID', 'GAME_DATE', 'MATCHUP', 'WL', 'MIN', 'PTS', 'FGM', 'FGA', 'FG_PCT', 'FG3M', 'FG3A', 'FG3_PCT', 'FTM', 'FTA', 'FT_PCT', 'OREB', 'DREB', 'REB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PLUS_MINUS']


In [83]:
print(games_vs_lakers.head())


  SEASON_ID     TEAM_ID TEAM_ABBREVIATION         TEAM_NAME     GAME_ID  \
0     22024  1610612742               DAL  Dallas Mavericks  0022400812   
1     22024  1610612742               DAL  Dallas Mavericks  0022400803   
2     22024  1610612742               DAL  Dallas Mavericks  0022400786   
3     22024  1610612742               DAL  Dallas Mavericks  0022400782   
4     22024  1610612742               DAL  Dallas Mavericks  0022400762   

    GAME_DATE      MATCHUP WL  MIN  PTS  ...  FT_PCT  OREB  DREB  REB  AST  \
0  2025-02-23    DAL @ GSW  L  240  102  ...   0.684     7    34   41   24   
1  2025-02-21  DAL vs. NOP  W  240  111  ...   0.778     8    39   47   28   
2  2025-02-13  DAL vs. MIA  W  242  118  ...   0.813    11    29   40   28   
3  2025-02-12  DAL vs. GSW  W  240  111  ...   0.800     9    30   39   20   
4  2025-02-10  DAL vs. SAC  L  265  128  ...   0.625     5    28   33   31   

   STL  BLK  TOV  PF  PLUS_MINUS  
0    7    4   18  22       -24.0  
1    8    

In [81]:
print(games_vs_lakers.info())  # See data types and available columns


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3830 entries, 0 to 3829
Data columns (total 28 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   SEASON_ID          3830 non-null   object 
 1   TEAM_ID            3830 non-null   int64  
 2   TEAM_ABBREVIATION  3830 non-null   object 
 3   TEAM_NAME          3830 non-null   object 
 4   GAME_ID            3830 non-null   object 
 5   GAME_DATE          3830 non-null   object 
 6   MATCHUP            3830 non-null   object 
 7   WL                 3828 non-null   object 
 8   MIN                3830 non-null   int64  
 9   PTS                3830 non-null   int64  
 10  FGM                3830 non-null   int64  
 11  FGA                3830 non-null   int64  
 12  FG_PCT             3829 non-null   float64
 13  FG3M               3830 non-null   int64  
 14  FG3A               3830 non-null   int64  
 15  FG3_PCT            3807 non-null   float64
 16  FTM                3830 

In [63]:
print("Available columns in games_vs_lakers:")
print(games_vs_lakers.columns.tolist())  # Display all column names


Available columns in games_vs_lakers:
['SEASON_ID', 'TEAM_ID', 'TEAM_ABBREVIATION', 'TEAM_NAME', 'GAME_ID', 'GAME_DATE', 'MATCHUP', 'WL', 'MIN', 'PTS', 'FGM', 'FGA', 'FG_PCT', 'FG3M', 'FG3A', 'FG3_PCT', 'FTM', 'FTA', 'FT_PCT', 'OREB', 'DREB', 'REB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PLUS_MINUS']


In [61]:
print(f"Number of rows in games_vs_lakers: {len(games_vs_lakers)}")


Number of rows in games_vs_lakers: 3830


In [59]:
print(games_vs_lakers.head())  # Check first few rows


  SEASON_ID     TEAM_ID TEAM_ABBREVIATION         TEAM_NAME     GAME_ID  \
0     22024  1610612742               DAL  Dallas Mavericks  0022400812   
1     22024  1610612742               DAL  Dallas Mavericks  0022400803   
2     22024  1610612742               DAL  Dallas Mavericks  0022400786   
3     22024  1610612742               DAL  Dallas Mavericks  0022400782   
4     22024  1610612742               DAL  Dallas Mavericks  0022400762   

    GAME_DATE      MATCHUP WL  MIN  PTS  ...  FT_PCT  OREB  DREB  REB  AST  \
0  2025-02-23    DAL @ GSW  L  240  102  ...   0.684     7    34   41   24   
1  2025-02-21  DAL vs. NOP  W  240  111  ...   0.778     8    39   47   28   
2  2025-02-13  DAL vs. MIA  W  242  118  ...   0.813    11    29   40   28   
3  2025-02-12  DAL vs. GSW  W  240  111  ...   0.800     9    30   39   20   
4  2025-02-10  DAL vs. SAC  L  265  128  ...   0.625     5    28   33   31   

   STL  BLK  TOV  PF  PLUS_MINUS  
0    7    4   18  22       -24.0  
1    8    

In [53]:
mavs_games = games_vs_lakers[games_vs_lakers['SEASON_ID_MAVS'].str.contains("DAL", na=False)]
lakers_games = games_vs_lakers[games_vs_lakers['SEASON_ID_LAKERS'].str.contains("LAL", na=False)]


In [55]:
from nba_api.stats.endpoints import leaguegamefinder

mavs_id = '1610612742'
lakers_id = '1610612747'

gamefinder = leaguegamefinder.LeagueGameFinder(team_id_nullable=mavs_id)
games_vs_lakers = gamefinder.get_data_frames()[0]  # Retrieve data

# Check available columns
print(games_vs_lakers.columns.tolist())
print(games_vs_lakers.head())


['SEASON_ID', 'TEAM_ID', 'TEAM_ABBREVIATION', 'TEAM_NAME', 'GAME_ID', 'GAME_DATE', 'MATCHUP', 'WL', 'MIN', 'PTS', 'FGM', 'FGA', 'FG_PCT', 'FG3M', 'FG3A', 'FG3_PCT', 'FTM', 'FTA', 'FT_PCT', 'OREB', 'DREB', 'REB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PLUS_MINUS']
  SEASON_ID     TEAM_ID TEAM_ABBREVIATION         TEAM_NAME     GAME_ID  \
0     22024  1610612742               DAL  Dallas Mavericks  0022400812   
1     22024  1610612742               DAL  Dallas Mavericks  0022400803   
2     22024  1610612742               DAL  Dallas Mavericks  0022400786   
3     22024  1610612742               DAL  Dallas Mavericks  0022400782   
4     22024  1610612742               DAL  Dallas Mavericks  0022400762   

    GAME_DATE      MATCHUP WL  MIN  PTS  ...  FT_PCT  OREB  DREB  REB  AST  \
0  2025-02-23    DAL @ GSW  L  240  102  ...   0.684     7    34   41   24   
1  2025-02-21  DAL vs. NOP  W  240  111  ...   0.778     8    39   47   28   
2  2025-02-13  DAL vs. MIA  W  242  118  ...   0.813   

In [57]:
print(games_vs_lakers.columns.tolist())

['SEASON_ID', 'TEAM_ID', 'TEAM_ABBREVIATION', 'TEAM_NAME', 'GAME_ID', 'GAME_DATE', 'MATCHUP', 'WL', 'MIN', 'PTS', 'FGM', 'FGA', 'FG_PCT', 'FG3M', 'FG3A', 'FG3_PCT', 'FTM', 'FTA', 'FT_PCT', 'OREB', 'DREB', 'REB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PLUS_MINUS']


In [147]:
from nba_api.stats.endpoints import leaguegamefinder
import pandas as pd

# Mavericks ID
mavs_id = 1610612742
gamefinder_mavs = leaguegamefinder.LeagueGameFinder(team_id_nullable=mavs_id)
mavs_games = gamefinder_mavs.get_data_frames()[0]

print(f"Mavericks games count: {len(mavs_games)}")
print(mavs_games.head())  # Display sample data


Mavericks games count: 3830
  SEASON_ID     TEAM_ID TEAM_ABBREVIATION         TEAM_NAME     GAME_ID  \
0     22024  1610612742               DAL  Dallas Mavericks  0022400812   
1     22024  1610612742               DAL  Dallas Mavericks  0022400803   
2     22024  1610612742               DAL  Dallas Mavericks  0022400786   
3     22024  1610612742               DAL  Dallas Mavericks  0022400782   
4     22024  1610612742               DAL  Dallas Mavericks  0022400762   

    GAME_DATE      MATCHUP WL  MIN  PTS  ...  FT_PCT  OREB  DREB  REB  AST  \
0  2025-02-23    DAL @ GSW  L  240  102  ...   0.684     7    34   41   24   
1  2025-02-21  DAL vs. NOP  W  240  111  ...   0.778     8    39   47   28   
2  2025-02-13  DAL vs. MIA  W  242  118  ...   0.813    11    29   40   28   
3  2025-02-12  DAL vs. GSW  W  240  111  ...   0.800     9    30   39   20   
4  2025-02-10  DAL vs. SAC  L  265  128  ...   0.625     5    28   33   31   

   STL  BLK  TOV  PF  PLUS_MINUS  
0    7    4   18 

In [149]:
# Lakers ID
lakers_id = 1610612747
gamefinder_lakers = leaguegamefinder.LeagueGameFinder(team_id_nullable=lakers_id)
lakers_games = gamefinder_lakers.get_data_frames()[0]

print(f"Lakers games count: {len(lakers_games)}")
print(lakers_games.head())  # Display sample data


Lakers games count: 4031
  SEASON_ID     TEAM_ID TEAM_ABBREVIATION           TEAM_NAME     GAME_ID  \
0     22024  1610612747               LAL  Los Angeles Lakers  0022400808   
1     22024  1610612747               LAL  Los Angeles Lakers  0022400796   
2     22024  1610612747               LAL  Los Angeles Lakers  0022400524   
3     22024  1610612747               LAL  Los Angeles Lakers  0022400781   
4     22024  1610612747               LAL  Los Angeles Lakers  0022400768   

    GAME_DATE      MATCHUP WL  MIN  PTS  ...  FT_PCT  OREB  DREB  REB  AST  \
0  2025-02-22    LAL @ DEN  W  239  123  ...   0.840     9    30   39   27   
1  2025-02-20    LAL @ POR  W  240  110  ...   0.920    11    37   48   20   
2  2025-02-19  LAL vs. CHA  L  239   97  ...   0.667    12    38   50   29   
3  2025-02-12    LAL @ UTA  L  240  119  ...   0.567    14    31   45   31   
4  2025-02-10  LAL vs. UTA  W  241  132  ...   0.944    10    34   44   28   

   STL  BLK  TOV  PF  PLUS_MINUS  
0   15  

In [151]:
# Filter Mavericks games where the opponent is the Lakers
mavs_vs_lakers = mavs_games[mavs_games['MATCHUP'].str.contains("LAL", na=False)]

# Filter Lakers games where the opponent is the Mavericks
lakers_vs_mavs = lakers_games[lakers_games['MATCHUP'].str.contains("DAL", na=False)]

print(f"Mavericks vs Lakers games count: {len(mavs_vs_lakers)}")
print(f"Lakers vs Mavericks games count: {len(lakers_vs_mavs)}")


Mavericks vs Lakers games count: 186
Lakers vs Mavericks games count: 186


In [155]:
games_vs_lakers = mavs_vs_lakers.merge(lakers_vs_mavs, on='GAME_DATE', suffixes=('_MAVS', '_LAKERS'))
print(f"Games after merging: {len(games_vs_lakers)}")
print(games_vs_lakers.head())


Games after merging: 186
  SEASON_ID_MAVS  TEAM_ID_MAVS TEAM_ABBREVIATION_MAVS    TEAM_NAME_MAVS  \
0          22024    1610612742                    DAL  Dallas Mavericks   
1          22023    1610612742                    DAL  Dallas Mavericks   
2          22023    1610612742                    DAL  Dallas Mavericks   
3          22023    1610612742                    DAL  Dallas Mavericks   
4          22022    1610612742                    DAL  Dallas Mavericks   

  GAME_ID_MAVS   GAME_DATE MATCHUP_MAVS WL_MAVS  MIN_MAVS  PTS_MAVS  ...  \
0   0022400504  2025-01-07  DAL vs. LAL       W       238       118  ...   
1   0022300577  2024-01-17    DAL @ LAL       L       240       110  ...   
2   0022300306  2023-12-12  DAL vs. LAL       W       239       127  ...   
3   0022300238  2023-11-22    DAL @ LAL       W       240       104  ...   
4   0022201054  2023-03-17    DAL @ LAL       W       240       111  ...   

   FT_PCT_LAKERS  OREB_LAKERS  DREB_LAKERS  REB_LAKERS  AST_LAKERS 

In [157]:
print(len(mavs_games))
print(len(lakers_games))

3830
4031


In [159]:
print(games_vs_lakers)

    SEASON_ID_MAVS  TEAM_ID_MAVS TEAM_ABBREVIATION_MAVS    TEAM_NAME_MAVS  \
0            22024    1610612742                    DAL  Dallas Mavericks   
1            22023    1610612742                    DAL  Dallas Mavericks   
2            22023    1610612742                    DAL  Dallas Mavericks   
3            22023    1610612742                    DAL  Dallas Mavericks   
4            22022    1610612742                    DAL  Dallas Mavericks   
..             ...           ...                    ...               ...   
181          21983    1610612742                    DAL  Dallas Mavericks   
182          21983    1610612742                    DAL  Dallas Mavericks   
183          21983    1610612742                    DAL  Dallas Mavericks   
184          21983    1610612742                    DAL  Dallas Mavericks   
185          21983    1610612742                    DAL  Dallas Mavericks   

    GAME_ID_MAVS   GAME_DATE MATCHUP_MAVS WL_MAVS  MIN_MAVS  PTS_MAVS  ... 

In [161]:
print(games_vs_lakers.columns.tolist())

['SEASON_ID_MAVS', 'TEAM_ID_MAVS', 'TEAM_ABBREVIATION_MAVS', 'TEAM_NAME_MAVS', 'GAME_ID_MAVS', 'GAME_DATE', 'MATCHUP_MAVS', 'WL_MAVS', 'MIN_MAVS', 'PTS_MAVS', 'FGM_MAVS', 'FGA_MAVS', 'FG_PCT_MAVS', 'FG3M_MAVS', 'FG3A_MAVS', 'FG3_PCT_MAVS', 'FTM_MAVS', 'FTA_MAVS', 'FT_PCT_MAVS', 'OREB_MAVS', 'DREB_MAVS', 'REB_MAVS', 'AST_MAVS', 'STL_MAVS', 'BLK_MAVS', 'TOV_MAVS', 'PF_MAVS', 'PLUS_MINUS_MAVS', 'SEASON_ID_LAKERS', 'TEAM_ID_LAKERS', 'TEAM_ABBREVIATION_LAKERS', 'TEAM_NAME_LAKERS', 'GAME_ID_LAKERS', 'MATCHUP_LAKERS', 'WL_LAKERS', 'MIN_LAKERS', 'PTS_LAKERS', 'FGM_LAKERS', 'FGA_LAKERS', 'FG_PCT_LAKERS', 'FG3M_LAKERS', 'FG3A_LAKERS', 'FG3_PCT_LAKERS', 'FTM_LAKERS', 'FTA_LAKERS', 'FT_PCT_LAKERS', 'OREB_LAKERS', 'DREB_LAKERS', 'REB_LAKERS', 'AST_LAKERS', 'STL_LAKERS', 'BLK_LAKERS', 'TOV_LAKERS', 'PF_LAKERS', 'PLUS_MINUS_LAKERS']


In [163]:
# Create a binary column: 1 if Mavericks won, 0 if Lakers won
games_vs_lakers['WINNER'] = games_vs_lakers['WL_MAVS'].apply(lambda x: 1 if x == 'W' else 0)

# Drop redundant columns
games_vs_lakers = games_vs_lakers.drop(columns=['WL_MAVS', 'WL_LAKERS', 'MATCHUP_MAVS', 'MATCHUP_LAKERS', 'GAME_DATE'])


In [245]:
# Define feature columns
features = [
    'FGM_MAVS', 'FGA_MAVS', 'FG_PCT_MAVS', 'FTM_MAVS', 'FTA_MAVS', 'FT_PCT_MAVS', 'OREB_MAVS', 'DREB_MAVS', 'REB_MAVS', 'AST_MAVS', 'STL_MAVS',  'TOV_MAVS', 'PF_MAVS', 
    'FGM_LAKERS', 'FGA_LAKERS', 'FG_PCT_LAKERS', 'FTM_LAKERS', 'FTA_LAKERS', 'FT_PCT_LAKERS', 'OREB_LAKERS', 'DREB_LAKERS', 'REB_LAKERS', 'AST_LAKERS', 'STL_LAKERS',  'TOV_LAKERS', 'PF_LAKERS',
]

# Define target variable
X = games_vs_lakers[features]
y = games_vs_lakers['WINNER']


In [247]:
from sklearn.model_selection import train_test_split

# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"Training set size: {len(X_train)}")
print(f"Testing set size: {len(X_test)}")


Training set size: 148
Testing set size: 38


In [249]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Initialize the model
model = LogisticRegression()

# Train the model
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)

# Evaluate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Model Accuracy: {accuracy * 100:.2f}%")


Model Accuracy: 92.11%


C:\ProgramData\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [251]:
print(X.isnull().sum())  # Check which columns have NaN values


FGM_MAVS         0
FGA_MAVS         0
FG_PCT_MAVS      0
FTM_MAVS         0
FTA_MAVS         0
FT_PCT_MAVS      0
OREB_MAVS        0
DREB_MAVS        0
REB_MAVS         0
AST_MAVS         0
STL_MAVS         0
TOV_MAVS         0
PF_MAVS          0
FGM_LAKERS       0
FGA_LAKERS       0
FG_PCT_LAKERS    0
FTM_LAKERS       0
FTA_LAKERS       0
FT_PCT_LAKERS    0
OREB_LAKERS      0
DREB_LAKERS      0
REB_LAKERS       0
AST_LAKERS       0
STL_LAKERS       0
TOV_LAKERS       0
PF_LAKERS        0
dtype: int64


In [253]:
X = X.dropna()
y = y.loc[X.index]  # Ensure target labels match the remaining rows


In [255]:
print(f"Remaining rows after dropping NaNs: {len(X)}")


Remaining rows after dropping NaNs: 186


In [257]:
# Split the dataset again (in case we dropped rows)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize and train the model
model = LogisticRegression()
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)

# Evaluate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Model Accuracy: {accuracy * 100:.2f}%")


Model Accuracy: 92.11%


C:\ProgramData\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [259]:
model = LogisticRegression(max_iter=500, solver='saga')  # 'saga' works well for larger datasets
model.fit(X_train_scaled, y_train)


LogisticRegression(max_iter=500, solver='saga')

In [261]:
import numpy as np

print("Max values per feature before scaling:")
print(np.max(X_train, axis=0))  # Print max values of each feature

print("Min values per feature before scaling:")
print(np.min(X_train, axis=0))  # Print min values of each feature


Max values per feature before scaling:
FGM_MAVS          53.000
FGA_MAVS         107.000
FG_PCT_MAVS        0.622
FTM_MAVS          38.000
FTA_MAVS          46.000
FT_PCT_MAVS        1.000
OREB_MAVS         27.000
DREB_MAVS         51.000
REB_MAVS          64.000
AST_MAVS          37.000
STL_MAVS          17.000
TOV_MAVS          26.000
PF_MAVS           33.000
FGM_LAKERS        58.000
FGA_LAKERS       112.000
FG_PCT_LAKERS      0.654
FTM_LAKERS        37.000
FTA_LAKERS        48.000
FT_PCT_LAKERS      0.955
OREB_LAKERS       25.000
DREB_LAKERS       48.000
REB_LAKERS        63.000
AST_LAKERS        39.000
STL_LAKERS        20.000
TOV_LAKERS        27.000
PF_LAKERS         32.000
dtype: float64
Min values per feature before scaling:
FGM_MAVS         28.000
FGA_MAVS         65.000
FG_PCT_MAVS       0.344
FTM_MAVS          6.000
FTA_MAVS          6.000
FT_PCT_MAVS       0.500
OREB_MAVS         3.000
DREB_MAVS        16.000
REB_MAVS         25.000
AST_MAVS         12.000
STL_MAVS         

In [264]:
from sklearn.preprocessing import RobustScaler

scaler = RobustScaler()  # Handles outliers better
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Train with robust-scaled data
model = LogisticRegression(max_iter=1000, solver='saga')
model.fit(X_train_scaled, y_train)


LogisticRegression(max_iter=1000, solver='saga')

In [266]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)


In [268]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Make predictions on the test set
y_pred = model.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Model Accuracy: {accuracy * 100:.2f}%")

# Display a classification report (Precision, Recall, F1-Score)
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

# Confusion Matrix (for better visualization)
print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred))


Model Accuracy: 81.58%

Classification Report:
              precision    recall  f1-score   support

           0       0.91      0.81      0.86        26
           1       0.67      0.83      0.74        12

    accuracy                           0.82        38
   macro avg       0.79      0.82      0.80        38
weighted avg       0.84      0.82      0.82        38


Confusion Matrix:
[[21  5]
 [ 2 10]]


In [270]:
import numpy as np

# Example stats for a future Mavericks vs. Lakers game
new_game_data = np.array([[110, 24, 45, 0.48, 14, 3,   # Mavericks stats
                           115, 26, 50, 0.50, 12, 5]]) # Lakers stats

# Scale the new game data
new_game_data_scaled = scaler.transform(new_game_data)

# Make the prediction
prediction = model.predict(new_game_data_scaled)

# Print the predicted winner
if prediction[0] == 1:
    print("🏀 Predicted Winner: Dallas Mavericks")
else:
    print("🏀 Predicted Winner: Los Angeles Lakers")


C:\ProgramData\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RobustScaler was fitted with feature names
  warnings.warn(


ValueError: X has 12 features, but RobustScaler is expecting 26 features as input.

In [272]:
print("Features used in training:")
print(X_train.columns.tolist())


Features used in training:
['FGM_MAVS', 'FGA_MAVS', 'FG_PCT_MAVS', 'FTM_MAVS', 'FTA_MAVS', 'FT_PCT_MAVS', 'OREB_MAVS', 'DREB_MAVS', 'REB_MAVS', 'AST_MAVS', 'STL_MAVS', 'TOV_MAVS', 'PF_MAVS', 'FGM_LAKERS', 'FGA_LAKERS', 'FG_PCT_LAKERS', 'FTM_LAKERS', 'FTA_LAKERS', 'FT_PCT_LAKERS', 'OREB_LAKERS', 'DREB_LAKERS', 'REB_LAKERS', 'AST_LAKERS', 'STL_LAKERS', 'TOV_LAKERS', 'PF_LAKERS']


In [285]:
import pandas as pd

# Ensure new_game_data has all the features in the same order
new_game_data = pd.DataFrame([[38, 83, 42, 18, 24, 82, 8, 16, 24, 16, 4, 9, 110,   # Mavericks stats
                               47, 70, 63, 12, 16, 90, 12, 20, 22,44, 23, 7, 16]], # Lakers stats
                              columns=X_train.columns)  # Match training features

# Scale the new game data
new_game_data_scaled = scaler.transform(new_game_data)

# Make the prediction
prediction = model.predict(new_game_data_scaled)

# Print the predicted winner
if prediction[0] == 1:
    print("🏀 Predicted Winner: Dallas Mavericks")
else:
    print("🏀 Predicted Winner: Los Angeles Lakers")
    


🏀 Predicted Winner: Los Angeles Lakers


C:\ProgramData\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
